<a href="https://colab.research.google.com/github/ananya-rm/Open-source-huggingface/blob/main/Zero_shot_audio_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers.utils import logging

In [2]:
logging.set_verbosity_error()

In [7]:
!pip install fsspec

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
from datasets import load_dataset

In [9]:
dataset = load_dataset("ashraq/esc50",
                 split="train[0:10]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/345 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


dataset_infos.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

(…)-00000-of-00002-2f1ab7b824ec751f.parquet:   0%|          | 0.00/387M [00:00<?, ?B/s]

(…)-00001-of-00002-27425e5c1846b494.parquet:   0%|          | 0.00/387M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
audio_sample = dataset[0]

In [11]:
audio_sample

{'filename': '1-100032-A-0.wav',
 'fold': 1,
 'target': 0,
 'category': 'dog',
 'esc10': True,
 'src_file': 100032,
 'take': 'A',
 'audio': {'path': None,
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 44100}}

In [12]:
from IPython.display import Audio as IPythonAudio

In [14]:
from transformers import pipeline

In [19]:
zero_shot_classifier = pipeline("zero-shot-audio-classification","laion/clap-htsat-unfused")

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

In [20]:
zero_shot_classifier.feature_extractor.sampling_rate

48000

In [21]:
audio_sample["audio"]["sampling_rate"]

44100

In [22]:
from datasets import Audio

In [23]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=zero_shot_classifier.feature_extractor.sampling_rate))

In [24]:
audio_sample = dataset[0]

In [25]:
audio_sample

{'filename': '1-100032-A-0.wav',
 'fold': 1,
 'target': 0,
 'category': 'dog',
 'esc10': True,
 'src_file': 100032,
 'take': 'A',
 'audio': {'path': None,
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 48000}}

In [26]:
candidate_labels = ["Sound of a dog",
                    "Sound of vacuum cleaner"]

In [29]:
zero_shot_classifier(audio_sample["audio"]["array"],
                     candidate_labels=candidate_labels)

[{'score': 0.9985589385032654, 'label': 'Sound of a dog'},
 {'score': 0.001441098633222282, 'label': 'Sound of vacuum cleaner'}]

In [30]:
candidate_labels = ["Sound of a child crying",
                    "Sound of vacuum cleaner",
                    "Sound of a bird singing",
                    "Sound of an airplane"]


In [31]:
zero_shot_classifier(audio_sample["audio"]["array"],candidate_labels=candidate_labels)

[{'score': 0.6172540783882141, 'label': 'Sound of a bird singing'},
 {'score': 0.21602477133274078, 'label': 'Sound of vacuum cleaner'},
 {'score': 0.1254723221063614, 'label': 'Sound of an airplane'},
 {'score': 0.04124874994158745, 'label': 'Sound of a child crying'}]